In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/newspaper-text-summarization-cnn-dailymail/cnn_dailymail/validation.csv
/kaggle/input/newspaper-text-summarization-cnn-dailymail/cnn_dailymail/train.csv
/kaggle/input/newspaper-text-summarization-cnn-dailymail/cnn_dailymail/test.csv


In [2]:
df=pd.read_csv("/kaggle/input/newspaper-text-summarization-cnn-dailymail/cnn_dailymail/train.csv")

In [3]:
!pip install contractions

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 9.4 MB/s eta 0:00:00


Helper functions

In [4]:
#used to regex clean
def re_clean(text):
    import re
    text = re.sub(r'https?:\/\/.*[\r\n]*', ' ', text, flags=re.MULTILINE)
    text = re.sub(r'\<a href', ' ', text)
    text = re.sub(r'&amp;', ' ', text) 
    text = re.sub(r'[_\-;%()|+&=*%:#$@\[\]/]', ' ', text)
    text = re.sub(r'<br />', ' ', text)
    text = re.sub(r'\'', ' ', text)
    text=re.sub(r'\n',' ',text)
    text=re.sub(' est ',' ',text)
    text=re.sub(r'[?!]','.',text)
    return text
#used to expand contractions
def expand(text):
    import contractions
    text=text.split()
    final=[]
    for word in text:
        try:
            final.append(contractions.fix(word)+" ") 
        except:
            final.append(word+" ")
            print(word)
    return "".join(final)
def remove(texts):
    final=[]
    for text in texts:
        sents=[]
        sentences=text.split(".")
        for sentence in sentences:
            if(len(sentence.split())>=5):
                sents.append(sentence+".")
        final.append("".join(sents))
    return final
def removeTag(texts):
    final=[]
    #removing cnn and est
    for text in texts:
        cnn=text.find("cnn")
        if(cnn!=-1 and cnn<len(text)//10):
            text=text[cnn+3:]
        found=False
        for i in range(2):
            est=text.find(" est,")
            if(est<len(text)//5 and est!=-1):
                text=text[est+5:]
                found=True
        fs=text.find(".")
        if(fs<20 and fs!=-1 and found):
            text=text[fs:]
        final.append(text)
    return final
        
    



In [5]:
texts=df.iloc[:,1]

In [6]:
new_texts=[]
for i in texts:
    new_texts.append(expand(re_clean(i)).lower())
finalcleaned=removeTag(remove(new_texts))

İbrahim
İbrahim
İbrahim


In [7]:
summaries=df.iloc[:,2]

In [8]:
new_summaries=[]
for i in summaries:
    new_summaries.append(expand(re_clean(i)).lower())

In [9]:
newdf=pd.DataFrame()
newdf["Texts"]=finalcleaned
newdf["Summaries"]=new_summaries

In [10]:
newdf.to_csv("Cleaned.csv")

In [11]:
newdf

,Texts,Summaries
0,. the bishop of the fargo catholic diocese in ...,"bishop john folda, of north dakota, is taking ..."
1,ralph mata was an internal affairs lieutenant...,criminal complaint cop used his role to help c...
2,a drunk driver who killed a young woman in a h...,"craig eccleston todd, 27, had drunk at least t..."
3,with a breezy sweep of his pen president vlad...,nina dos santos says europe must be ready to a...
4,fleetwood are the only team still to have a 10...,fleetwood top of league one after 2 0 win at s...
...,...,...
287108,former first daughter chelsea clinton has ind...,chelsea clinton said question of running for o...
287109,an apologetic vanilla ice has given his first ...,"vanilla ice, 47 real name robert van winkle wa..."
287110,america s most lethal sniper claimed he wished...,america s most lethal sniper made comment in i...
287111,. israel has been invaded by a swarm of more t...,a swarm of more than one million has crossed b...


In [12]:

def words(texts):
    import re
    setofwords=set()
    for text in texts:
        text=re.sub(r'[.,]',' ',text)
        words=text.split()
        for word in words:
            setofwords.add(word)
    return setofwords
swords=words(finalcleaned)



IDF calculation

In [13]:
import re
import math
freq={}
for i in swords:
    freq[i]=0
for text in finalcleaned:
    text=re.sub(r'[,.]',' ',text)
    ws=set(text.split())
    for i in ws:
        if(i in freq):
            freq[i]+=1
n=len(finalcleaned)
idf={}
for i in freq:
    idf[i]=math.log(n/freq[i])

In [14]:
import pickle
with open("idf.pkl","wb") as f:
    pickle.dump(idf,f)